In [1]:
import sys
!{sys.executable} -m pip install sparqlwrapper

In [2]:
import requests
from string import Template
from SPARQLWrapper import SPARQLWrapper, JSON, TURTLE
from urllib.parse import quote
from os.path import join

In [3]:
endpoint = 'http://localhost:8887/blazegraph/sparql'
prefixes = """
PREFIX ldp: <http://www.w3.org/ns/ldp#>
PREFIX Platform: <http://www.metaphacts.com/ontologies/platform#>
"""
outputDir = '../output'

In [4]:
def sparqlResultToDict(results):
    rows = []
    for result in results["results"]["bindings"]:
        row = {}
        for key in list(result.keys()):
            row[key] = result[key]["value"]
        rows.append(row)
    return rows

In [5]:
sparql = SPARQLWrapper(endpoint)
sparql.setReturnFormat(JSON)

In [6]:
graphsQuery = prefixes + """
SELECT DISTINCT ?g WHERE {
    GRAPH ?g {
        Platform:formContainer ldp:contains ?container .
    }
}
"""

In [7]:
sparql.setQuery(graphsQuery)
results = sparql.query().convert()

In [8]:
graphs = [d['g'] for d in sparqlResultToDict(results)]

In [9]:
constructTemplate = Template(prefixes + """
CONSTRUCT {
    ?s ?p ?o .
} WHERE {
  GRAPH <$graph> {
      ?s ?p ?o
  }
}
""")


In [10]:
graphs

['https://resource.swissartresearch.net/artwork/zbz-010055116/proposition/eb755691-b70b-4bb4-9acb-6f42f02e9543/container/context',
 'http://www.researchspace.org/ontology/ImageRegion/11e53146-bbc9-4ff1-91b9-30edda42e03d/container/context']

In [11]:
headers = {
    "Accept": "application/x-trig"
}

queryTemplate = Template("""CONSTRUCT {
        ?s ?p ?o
        } FROM <$graph> WHERE {
            ?s ?p ?o .
        }""")

for graph in graphs:
    data = {
        "query": queryTemplate.substitute(graph=graph)
    }
    try:
        response = requests.get(endpoint, headers=headers, params=data)
    except:
        print("Could not fetch data")
    if response:
        output = response.content.decode().replace("{", "<" + graph + "> {")
        filename = quote(graph, safe='')
        with open(join(outputDir, filename), 'w') as f:
            f.write(output)